# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Загрузим и посмотрим данные

In [2]:
gd0 = pd.read_csv('/datasets/geo_data_0.csv')

gd0.info()
gd0.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
gd1 = pd.read_csv('/datasets/geo_data_1.csv')

gd1.info()
gd1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [4]:
gd2 = pd.read_csv('/datasets/geo_data_2.csv')

gd2.info()
gd2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


## Обучение и проверка модели

Создадим функцию для обучения модели, используя scaler transform для уравновешивания переменных

In [5]:
def model_lin(data, name):
    
    features = data.drop(['id','product'], axis=1)
    target = data['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    features_train, features_test, target_train, target_test = train_test_split(features_train, target_train, test_size=0.20, random_state=12345)
    scaler = StandardScaler()
    numeric = features.columns
    scaler.fit(features_train)
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    predictions_valid = pd.Series(predictions_valid, index = target_valid.index)
    
    print(name)
    print('MAE:', mean_absolute_error(target_valid, predictions_valid))
    print('MSE:', mean_squared_error(target_valid, predictions_valid))
    print('R2:', r2_score(target_valid, predictions_valid))
    print('RMSE', mean_squared_error(target_valid, predictions_valid) ** 0.5)
    print('Mean predict:', predictions_valid.mean())
    
    return target_valid, predictions_valid, features_test, target_test

Сравним результаты между тремя локациями

In [6]:
target_valid0, predictions_valid0, features_test0, target_test0 = model_lin(gd0, 'Зона 0')

Зона 0
MAE: 30.920634271085937
MSE: 1412.197321438156
R2: 0.27995117699602534
RMSE 37.57921395450091
Mean predict: 92.60257696973629


/tmp/ipykernel_48/1121778781.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_valid[numeric] = scaler.transform(features_valid[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [7]:
target_valid1, predictions_valid1, features_test1, target_test1 = model_lin(gd1, 'Зона 1')

/tmp/ipykernel_48/1121778781.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_valid[numeric] = scaler.transform(features_valid[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

Зона 1
MAE: 0.7188864626797543
MSE: 0.7976884434903839
R2: 0.9996233685562581
RMSE 0.8931340568416277
Mean predict: 68.7290942602421


In [8]:
target_valid2, predictions_valid2, features_test2, target_test2 = model_lin(gd2, 'Зона 2')

/tmp/ipykernel_48/1121778781.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_valid[numeric] = scaler.transform(features_valid[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

Зона 2
MAE: 32.79473884393918
MSE: 1602.4103574911571
R2: 0.2052313274308052
RMSE 40.03011812986763
Mean predict: 94.9577013294152


## Подготовка к расчёту прибыли

3.1 Ключевые значения

In [9]:
NUMBER_OF_WELLS = 500
BEST_WELLS = 200
BUDGET = 10 ** 10
INCOME_PER_UNIT = 450*10**3
MAX_PROB_LOSS = 2.5 #%

3.2 Рассчитаем минимальный объем скважины

In [10]:
min_prod = (BUDGET / INCOME_PER_UNIT) / 200
print(f"Минимальный объем скважины {min_prod:.2f}")

Минимальный объем скважины 111.11


Минимальный объем меньше чем средний объем скважин

In [11]:
predicted_0 = pd.Series(predictions_valid0, index = target_valid0.index)
predicted_1 = pd.Series(predictions_valid1, index = target_valid1.index)
predicted_2 = pd.Series(predictions_valid2, index = target_valid2.index)

In [70]:
def profits(target, predict):
    sort_predict = predict.sort_values(ascending=False)
    select_target = target[sort_predict.index][:BEST_WELLS]
    income = (select_target.sum() * INCOME_PER_UNIT - BUDGET)
    #print(len(sort_predict))
    #print(len(top_200))
    #print(len(select_target))
    return income

## Расчёт прибыли и рисков 

4.1 выбираем скважины с максимальными значениями

In [71]:
print(f'Прибыль с лучших скважин в 0 регионе: {profits(target_valid0, predicted_0):.2f} рублей')
print(f'Прибыль с лучших скважин в 1 регионе: {profits(target_valid1, predicted_1):.2f} рублей')
print(f'Прибыль с лучших скважин в 2 регионе: {profits(target_valid2, predicted_2):.2f} рублей')


Прибыль с лучших скважин в 0 регионе: 3320826043.14 рублей
Прибыль с лучших скважин в 1 регионе: 2415086696.68 рублей
Прибыль с лучших скважин в 2 регионе: 2710349963.60 рублей


Выполним процедуру boostrap

In [72]:
state = np.random.RandomState(12345)

def boostrape(target, predict, name='нет данных', count=500):
    profit_list = []
    
    for i in range(1000):
        predict_sample = predict.sample(count, replace=True, random_state = state)
        target_sample = target[predict_sample.index]
        profit_list.append(profits(target_sample, predict_sample))
    profit_list = pd.Series(profit_list)
    
    mean_profit = profit_list.mean()
    loss = (profit_list < 0).mean()
    
    lower = profit_list.quantile(0.025)
    higher = profit_list.quantile(0.975)
    
    print(f'Средняя прибыль для {name}: {mean_profit:.2f}')
    print(f'Доверительный интервал 95% для {name}: {int(lower)} - {int(higher)}')
    print(f'Вероятность убытка для {name}: {loss}')
    

In [76]:
print(boostrape(target_valid0, predicted_0, 'Регион 0'))

Средняя прибыль для Регион 0: 425186782.10
Доверительный интервал 95% для Регион 0: -102857913 - 976556627
Вероятность убытка для Регион 0: 0.052
None


In [77]:
print(boostrape(target_valid1, predicted_1, 'Регион 1'))

Средняя прибыль для Регион 1: 504737996.37
Доверительный интервал 95% для Регион 1: 77394602 - 932883332
Вероятность убытка для Регион 1: 0.009
None


In [78]:
print(boostrape(target_valid2, predicted_2, 'Регион 2'))

Средняя прибыль для Регион 2: 410052078.35
Доверительный интервал 95% для Регион 2: -195356879 - 961291553
Вероятность убытка для Регион 2: 0.091
None


Вывод, лучший регион для расположения наших скважин, это первый регион, у него средняя прибыль больше чем у других регионов, а также уровен риска минимальный. Этот регион лучше остальный подойдет под размещение скважин